In [24]:
import torch
from torch import nn
from myModel import SRResNet
from myDataset import dataSet
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
from mydataList1 import genDataList
import numpy as np

from PIL import Image
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

In [25]:
genDataList(t_sets=['./Set5', './Set14'], dataType='test', output = './output')
#genDataList(train_sets=['./BSDS100', './Urban100'], test_sets=['./Set5', './Set14'], output = './output')

Loading datasets....
size of test set 19 

Load complete
all the things save in output folder


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataPath = './output/'
rgb_weights = torch.FloatTensor([65.481, 128.553, 24.966]).to(device)

import time


def runEval():
    SRnet = SRResNet()
    SRnet =  SRnet.to(device)
    loss_fn = nn.MSELoss()
    #loss_function = nn.MSELoss().to(device) 
    
    if torch.cuda.is_available():
        SRnet = nn.DataParallel(SRnet, device_ids=list(range(2)))
    
    test_dataset = dataSet(dataPath=dataPath, dataType='test')
    train_loader = DataLoader(test_dataset,batch_size=64)
    
    PSNRs = 0.0
    SSIMs = 0.0
    sumP = 0.0
    SRnet.eval()
    start_t = time.time()
        
    with torch.no_grad():
        for i,(result, cropResult) in tqdm(enumerate(train_loader,1)):
            result = result.to(device)
            cropResult = cropResult.to(device)
            
            #forward 
            next_imgs = SRnet(cropResult)
            
            #calculate PSNR and SSIM
            c_next = next_imgs.squeeze(0)
            c_result = result.squeeze(0)
            
            psnr = peak_signal_noise_ratio(c_result.cpu().numpy(), c_next.cpu().numpy(),data_range=255.)
            
            #ssim = structural_similarity(result.cpu().numpy(), c_next.type(torch.uint8).detach().cpu().numpy(),in_size=None,data_range=255.)
    
            count = 1
            sumP += psnr
            #sumS += ssim
            count+=1
            avgP = sumP/count
            #avgS = sumS/count
    
    print(avgP)
    #print('SSIM  {ssims.avg:.3f}'.format(ssims=SSIMs))
    print('平均单张样本用时  {:.3f} 秒'.format((time.time()-start_t)/len(test_dataset)))



In [27]:
runEval()

1it [00:00,  1.27it/s]

26.63608307761372
平均单张样本用时  0.042 秒


In [28]:
imgPath = './output/me4.jpg'
scaleNum = 4
from torchvision import transforms

std = [0.229, 0.224, 0.225]
mean = [0.485, 0.456, 0.406]

std_t = torch.FloatTensor(std)
std_t = std_t.unsqueeze(1).unsqueeze(2)
mean_t = torch.FloatTensor(mean)
mean_t = mean_t.unsqueeze(1).unsqueeze(2)

def runTest():
    SRnet = SRResNet()
    SRnet =  SRnet.to(device)
    SRnet.eval()
    
    img = Image.open(imgPath, mode='r')
    img = img.convert('RGB')
    
    newWidth = img.width * scaleNum
    newHeihgt = img.height * scaleNum
    Bicubic_img = img.resize((int(newWidth),int(newHeihgt)),Image.BICUBIC)
    Bicubic_img.save('./output/result.jpg')
    
    transform = transforms.Compose([transforms.ToTensor()]) 
    lowRate = transform(img)
    
    #if lowRate.ndimension() == 3:
        #print('adfa')
    lowRate = (lowRate-mean_t)/std_t
    lowRate.unsqueeze_(0)
    
    start_t = time.time()
    lowRate = lowRate.to(device)
    
    with torch.no_grad():
        result = SRnet(lowRate).squeeze(0).cpu().detach()
        result = transforms.ToPILImage()(result).convert('RGB')
        result.save('./output/result_srr.jpg')

In [29]:
runTest()

C:\Users\14044\AppData\Local\Temp\ipykernel_14784\3561058132.py:23: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  Bicubic_img = img.resize((int(newWidth),int(newHeihgt)),Image.BICUBIC)
